In [ ]:
#導入postgresql套件
import psycopg2 
#建立連線
conn = psycopg2.connect(database="mydb", user="postgres", password="M444y335p336", host="127.0.0.1", port="5432")
#連線成功列印Opened database successfully
print("Opened database successfully")

Opened database successfully


In [7]:
#導入postgresql套件
import psycopg2
#環境變數儲存器(可先將環境變數儲存在.env的檔案中，透過load_dotenv將環境變數導入)
from dotenv import load_dotenv
#呼叫系統檔案或程式
import os
load_dotenv()

#psycopg2.connect建立DB連線，os.environ取得相關環境變數
conn = psycopg2.connect(host=os.environ['POSTGRE_HOST'],database=os.environ['POSTGRE_DATABASE'],user=os.environ['POSTGRE_USER'],password=os.environ['POSTGRE_PASSWORD'])
#開啟管理資源分配器(Context Manager)，DB連線時開啟，結束時關閉
with conn:
    print("Opened database successfully")

Opened database successfully
